### 数据格式

In [1]:
!head -3 /media/jw/5dccc50e-7c13-4186-bf6b-d894f02410be/wbb/result/qq_5000_user_data/qq_5000_user_dpi/3days/20131228/part-19143

['2013-12-28 23:40:34', '8613770773315', '111.30.132.39', 'web', 'wspeed.qq.com', 'qq', '-', '1087', '3', '3', '8', '2148', '-1']
['2013-12-28 23:41:41', '8613770773315', '111.30.132.38', 'web', 'wspeed.qq.com', 'qq', '-', '1621', '5', '5', '8', '2148', '-1']
['2013-12-28 23:43:52', '8613770773315', '111.30.132.39', 'web', 'wspeed.qq.com', 'qq', '-', '1140', '5', '5', '8', '2148', '-1']


In [2]:
# 过滤出需要用户和字段的三天数据
qq_5000_user_dpi_day1_f = "/media/jw/5dccc50e-7c13-4186-bf6b-d894f02410be/wbb/result/qq_5000_user_data/qq_5000_user_dpi/3days/20131228/part-*"
qq_5000_user_dpi_day2_f = "/media/jw/5dccc50e-7c13-4186-bf6b-d894f02410be/wbb/result/qq_5000_user_data/qq_5000_user_dpi/3days/20131229/part-*"
qq_5000_user_dpi_day3_f = "/media/jw/5dccc50e-7c13-4186-bf6b-d894f02410be/wbb/result/qq_5000_user_data/qq_5000_user_dpi/3days/20131230/part-*"

qq_5000_user_dpi_day4_f = "/media/jw/5dccc50e-7c13-4186-bf6b-d894f02410be/wbb/result/qq_5000_user_data/qq_5000_user_dpi/20131231/part-*"

qq_5000_user_dpi_3days_f = "/media/jw/5dccc50e-7c13-4186-bf6b-d894f02410be/wbb/result/qq_5000_user_data/qq_5000_user_dpi/3days/*/part-*"

In [35]:
qq_5000_user_dpi_day1 = sc.textFile(qq_5000_user_dpi_day1_f).coalesce(50)  # repartition
qq_5000_user_dpi_day2 = sc.textFile(qq_5000_user_dpi_day2_f).coalesce(50)
qq_5000_user_dpi_day3 = sc.textFile(qq_5000_user_dpi_day3_f).coalesce(50)

qq_5000_user_dpi_day4 = sc.textFile(qq_5000_user_dpi_day4_f).coalesce(50)

qq_5000_user_dpi_3days = sc.textFile(qq_5000_user_dpi_3days_f).coalesce(50)  

### 获取特征

In [6]:
# 路径
base_path = "/media/jw/5dccc50e-7c13-4186-bf6b-d894f02410be/wbb/result/qq_prediction/"

#### user_3digits

In [16]:
qq_user_phone_3digits = sc.textFile(base_path + "qq_user_phone_5000/part-*").map(lambda x: (x, x[2:5]))
qq_user_phone_3digits.take(2)

[('8615195898003', '151'), ('8615905145554', '159')]

In [17]:
all_3digits = list(set(qq_user_phone_3digits.map(lambda x: x[1]).collect()))
print(len(all_3digits), all_3digits)

18 ['188', '134', '158', '182', '157', '159', '152', '147', '184', '137', '183', '138', '135', '150', '151', '187', '136', '139']


In [21]:
def get_feature_vector(feature, all_feature):
    feature_vector = [0 for i in range(len(all_feature))]
    feature_vector[all_feature.index(feature)] = 1
    return feature_vector

In [22]:
qq_user_phone_3digits_vec = qq_user_phone_3digits.map(lambda x: (x[0], get_feature_vector(x[1], all_3digits)))
qq_user_phone_3digits_vec.take(2)

[('8615195898003', [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0]),
 ('8615905145554', [0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])]

In [25]:
qq_user_phone_3digits_vec.saveAsTextFile(base_path + "qq_user_phone_3digits")

#### user_brand

In [54]:
# 计算过程见feature_1.ipynb
user_brand_vec = sc.textFile(base_path + "user_brand_vec/part-*").map(lambda x: eval(x))

In [56]:
print(user_brand_vec.take(2))

[('8615951423477', [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]), ('8613913928726', [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])]


In [34]:
user_brand_vec.count()

5000

In [2]:
len([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

63

#### user_activetime: 用户活跃时间,每天24小时每小时的记录数

In [74]:
from collections import Counter
import numpy as np

In [62]:
user_activetime_day1 = qq_5000_user_dpi_day1.map(lambda x: (eval(x)[1], eval(x)[0][-8:-6])).groupByKey().mapValues(list)\
.map(lambda x: (x[0], dict(Counter(x[1]))))
user_activetime_day2 = qq_5000_user_dpi_day2.map(lambda x: (eval(x)[1], eval(x)[0][-8:-6])).groupByKey().mapValues(list)\
.map(lambda x: (x[0], dict(Counter(x[1]))))
user_activetime_day3 = qq_5000_user_dpi_day3.map(lambda x: (eval(x)[1], eval(x)[0][-8:-6])).groupByKey().mapValues(list)\
.map(lambda x: (x[0], dict(Counter(x[1]))))

In [304]:
# 由于‘day4’已经被定义为前三天的取平均，所以第4天DPI中提取的数据被定义为‘day5‘，以下'day5'均是指数据是来自第4天的
user_activetime_day5 = qq_5000_user_dpi_day4.map(lambda x: (eval(x)[1], eval(x)[0][-8:-6])).groupByKey().mapValues(list)\
.map(lambda x: (x[0], dict(Counter(x[1]))))

In [63]:
print(user_activetime_day1.count(), user_activetime_day1.take(2))

5000 [('8613851429132', {'12': 4, '14': 5, '11': 10}), ('8613913395739', {'12': 2, '23': 7, '17': 6, '11': 2, '13': 9, '16': 1, '06': 1, '08': 1, '09': 4, '21': 7})]


In [64]:
def get_feature(feature_dict, all_feature):
    res = [0 for i in range(len(all_feature))]
    for i in range(len(all_feature)):
        res[i] = feature_dict.get(all_feature[i],0)
    return res

In [65]:
time_hour = ['00', '01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', '21', '22', '23']

In [66]:
user_activetime_day1_vec = user_activetime_day1.map(lambda x: (x[0], get_feature(x[1], time_hour)))
user_activetime_day2_vec = user_activetime_day2.map(lambda x: (x[0], get_feature(x[1], time_hour)))
user_activetime_day3_vec = user_activetime_day3.map(lambda x: (x[0], get_feature(x[1], time_hour)))

In [305]:
user_activetime_day5_vec = user_activetime_day5.map(lambda x: (x[0], get_feature(x[1], time_hour)))

In [70]:
user_activetime_day1_vec.take(2)

[('8613851429132',
  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 10, 4, 0, 5, 0, 0, 0, 0, 0, 0, 0, 0, 0]),
 ('8613913395739',
  [0, 0, 0, 0, 0, 0, 1, 0, 1, 4, 0, 2, 2, 9, 0, 0, 1, 6, 0, 0, 0, 7, 0, 7])]

In [71]:
user_activetime_day1_vec.coalesce(2).saveAsTextFile(base_path + "user_activetime/day1/")
user_activetime_day2_vec.coalesce(2).saveAsTextFile(base_path + "user_activetime/day2/")
user_activetime_day3_vec.coalesce(2).saveAsTextFile(base_path + "user_activetime/day3/")

In [306]:
user_activetime_day5_vec.coalesce(2).saveAsTextFile(base_path + "user_activetime/day5/")

In [81]:
# 第4天的根据前三天的取平均
user_activetime_day4_vec = sc.union([user_activetime_day1_vec, user_activetime_day2_vec, user_activetime_day3_vec])\
.groupByKey().mapValues(list).map(lambda x: (x[0], list(np.mean(np.array(x[1]), axis=0, dtype=np.int16))))

In [82]:
user_activetime_day4_vec.take(3)

[('8613851429132',
  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3, 1, 0, 4, 0, 0, 0, 0, 13, 2, 0, 0, 0]),
 ('8615722826043',
  [0, 0, 0, 0, 0, 0, 0, 6, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 10, 6, 0, 1]),
 ('8618260098095',
  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 8, 21])]

In [83]:
user_activetime_day4_vec.coalesce(2).saveAsTextFile(base_path + "user_activetime/day4/")

#### user_diversity: 用户每天跟几个qq服务器发生连接

In [105]:
user_diversity_day1_vec = qq_5000_user_dpi_day1.map(lambda x: (eval(x)[1], eval(x)[2])).groupByKey().mapValues(set)\
.map(lambda x: (x[0], [len(x[1])]))
user_diversity_day2_vec = qq_5000_user_dpi_day2.map(lambda x: (eval(x)[1], eval(x)[2])).groupByKey().mapValues(set)\
.map(lambda x: (x[0], [len(x[1])]))
user_diversity_day3_vec = qq_5000_user_dpi_day3.map(lambda x: (eval(x)[1], eval(x)[2])).groupByKey().mapValues(set)\
.map(lambda x: (x[0], [len(x[1])]))

In [307]:
user_diversity_day5_vec = qq_5000_user_dpi_day4.map(lambda x: (eval(x)[1], eval(x)[2])).groupByKey().mapValues(set)\
.map(lambda x: (x[0], [len(x[1])]))

In [308]:
user_diversity_day5_vec.coalesce(2).saveAsTextFile(base_path + "user_diversity/day5/")

In [106]:
user_diversity_day2_vec.take(2)

[('8613851429132', [3]), ('8615722826043', [1])]

In [107]:
user_diversity_day4_vec = sc.union([user_diversity_day1_vec, user_diversity_day2_vec, user_diversity_day3_vec])\
.groupByKey().mapValues(list).map(lambda x: (x[0], list(np.mean(np.array(x[1]), axis=0, dtype=np.int16))))

In [108]:
user_diversity_day4_vec.take(2)

[('8613851429132', [6]), ('8615722826043', [3])]

In [109]:
user_diversity_day1_vec.coalesce(2).saveAsTextFile(base_path + "user_diversity/day1/")
user_diversity_day2_vec.coalesce(2).saveAsTextFile(base_path + "user_diversity/day2/")
user_diversity_day3_vec.coalesce(2).saveAsTextFile(base_path + "user_diversity/day3/")
user_diversity_day4_vec.coalesce(2).saveAsTextFile(base_path + "user_diversity/day4/")

#### user_indegree: 用户入度，每天跟qq服务器发生连接的次数

In [98]:
user_indegree_day1_vec = qq_5000_user_dpi_day1.map(lambda x: (eval(x)[1], eval(x)[2])).groupByKey().mapValues(list)\
.map(lambda x: (x[0], [len(x[1])]))
user_indegree_day2_vec = qq_5000_user_dpi_day2.map(lambda x: (eval(x)[1], eval(x)[2])).groupByKey().mapValues(list)\
.map(lambda x: (x[0], [len(x[1])]))
user_indegree_day3_vec = qq_5000_user_dpi_day3.map(lambda x: (eval(x)[1], eval(x)[2])).groupByKey().mapValues(list)\
.map(lambda x: (x[0], [len(x[1])]))

In [309]:
user_indegree_day5_vec = qq_5000_user_dpi_day4.map(lambda x: (eval(x)[1], eval(x)[2])).groupByKey().mapValues(list)\
.map(lambda x: (x[0], [len(x[1])]))

In [310]:
user_indegree_day5_vec.coalesce(2).saveAsTextFile(base_path + "user_indegree/day5/")

In [99]:
user_indegree_day2_vec.take(2)

[('8613851429132', [7]), ('8615722826043', [22])]

In [102]:
user_indegree_day4_vec = sc.union([user_indegree_day1_vec, user_indegree_day2_vec, user_indegree_day3_vec])\
.groupByKey().mapValues(list).map(lambda x: (x[0], list(np.mean(np.array(x[1]), axis=0, dtype=np.int16))))

In [103]:
user_indegree_day4_vec.take(2)

[('8613851429132', [24]), ('8615722826043', [29])]

In [104]:
user_indegree_day1_vec.coalesce(2).saveAsTextFile(base_path + "user_indegree/day1/")
user_indegree_day2_vec.coalesce(2).saveAsTextFile(base_path + "user_indegree/day2/")
user_indegree_day3_vec.coalesce(2).saveAsTextFile(base_path + "user_indegree/day3/")
user_indegree_day4_vec.coalesce(2).saveAsTextFile(base_path + "user_indegree/day4/")

#### user_in_strength: 用户每天下行报文数

In [110]:
user_in_strength_day1_vec = qq_5000_user_dpi_day1.map(lambda x: (eval(x)[1], int(eval(x)[9]))).groupByKey().mapValues(list)\
.map(lambda x: (x[0], [sum(x[1])]))
user_in_strength_day2_vec = qq_5000_user_dpi_day2.map(lambda x: (eval(x)[1], int(eval(x)[9]))).groupByKey().mapValues(list)\
.map(lambda x: (x[0], [sum(x[1])]))
user_in_strength_day3_vec = qq_5000_user_dpi_day3.map(lambda x: (eval(x)[1], int(eval(x)[9]))).groupByKey().mapValues(list)\
.map(lambda x: (x[0], [sum(x[1])]))

In [311]:
user_in_strength_day5_vec = qq_5000_user_dpi_day4.map(lambda x: (eval(x)[1], int(eval(x)[9]))).groupByKey().mapValues(list)\
.map(lambda x: (x[0], [sum(x[1])]))

In [312]:
user_in_strength_day5_vec.coalesce(2).saveAsTextFile(base_path + "user_in_strength/day5/")

In [111]:
user_in_strength_day2_vec.take(2)

[('8613851429132', [34]), ('8615722826043', [106])]

In [112]:
user_in_strength_day4_vec = sc.union([user_in_strength_day1_vec, user_in_strength_day2_vec, user_in_strength_day3_vec])\
.groupByKey().mapValues(list).map(lambda x: (x[0], list(np.mean(np.array(x[1]), axis=0, dtype=np.int16))))

In [113]:
user_in_strength_day4_vec.take(2)

[('8613851429132', [109]), ('8615722826043', [174])]

In [114]:
user_in_strength_day1_vec.coalesce(2).saveAsTextFile(base_path + "user_in_strength/day1/")
user_in_strength_day2_vec.coalesce(2).saveAsTextFile(base_path + "user_in_strength/day2/")
user_in_strength_day3_vec.coalesce(2).saveAsTextFile(base_path + "user_in_strength/day3/")
user_in_strength_day4_vec.coalesce(2).saveAsTextFile(base_path + "user_in_strength/day4/")

#### user_out_strength: 用户上行报文数

In [115]:
user_out_strength_day1_vec = qq_5000_user_dpi_day1.map(lambda x: (eval(x)[1], int(eval(x)[8]))).groupByKey().mapValues(list)\
.map(lambda x: (x[0], [sum(x[1])]))
user_out_strength_day2_vec = qq_5000_user_dpi_day2.map(lambda x: (eval(x)[1], int(eval(x)[8]))).groupByKey().mapValues(list)\
.map(lambda x: (x[0], [sum(x[1])]))
user_out_strength_day3_vec = qq_5000_user_dpi_day3.map(lambda x: (eval(x)[1], int(eval(x)[8]))).groupByKey().mapValues(list)\
.map(lambda x: (x[0], [sum(x[1])]))

In [313]:
user_out_strength_day5_vec = qq_5000_user_dpi_day4.map(lambda x: (eval(x)[1], int(eval(x)[8]))).groupByKey().mapValues(list)\
.map(lambda x: (x[0], [sum(x[1])]))

In [315]:
user_out_strength_day5_vec.coalesce(2).saveAsTextFile(base_path + "user_out_strength/day5/")

In [116]:
user_out_strength_day4_vec = sc.union([user_out_strength_day1_vec, user_out_strength_day2_vec, user_out_strength_day3_vec])\
.groupByKey().mapValues(list).map(lambda x: (x[0], list(np.mean(np.array(x[1]), axis=0, dtype=np.int16))))

In [117]:
user_out_strength_day1_vec.coalesce(2).saveAsTextFile(base_path + "user_out_strength/day1/")
user_out_strength_day2_vec.coalesce(2).saveAsTextFile(base_path + "user_out_strength/day2/")
user_out_strength_day3_vec.coalesce(2).saveAsTextFile(base_path + "user_out_strength/day3/")
user_out_strength_day4_vec.coalesce(2).saveAsTextFile(base_path + "user_out_strength/day4/")

#### user_duration: 用户每天和qq服务器之间的连接时长

In [343]:
user_duration_day1_vec = qq_5000_user_dpi_day1.map(lambda x: (eval(x)[1], int(eval(x)[7]))).groupByKey().mapValues(list)\
.map(lambda x: (x[0], [sum(x[1])]))
user_duration_day2_vec = qq_5000_user_dpi_day2.map(lambda x: (eval(x)[1], int(eval(x)[7]))).groupByKey().mapValues(list)\
.map(lambda x: (x[0], [sum(x[1])]))
user_duration_day3_vec = qq_5000_user_dpi_day3.map(lambda x: (eval(x)[1], int(eval(x)[7]))).groupByKey().mapValues(list)\
.map(lambda x: (x[0], [sum(x[1])]))

In [344]:
user_duration_day5_vec = qq_5000_user_dpi_day4.map(lambda x: (eval(x)[1], int(eval(x)[7]))).groupByKey().mapValues(list)\
.map(lambda x: (x[0], [sum(x[1])]))

In [346]:
user_duration_day1_vec.coalesce(2).saveAsTextFile(base_path + "user_duration/day1/")
user_duration_day2_vec.coalesce(2).saveAsTextFile(base_path + "user_duration/day2/")
user_duration_day3_vec.coalesce(2).saveAsTextFile(base_path + "user_duration/day3/")
user_duration_day5_vec.coalesce(2).saveAsTextFile(base_path + "user_duration/day5/")

In [347]:
user_duration_day4_vec = sc.union([user_duration_day1_vec, user_duration_day2_vec, user_duration_day3_vec])\
.groupByKey().mapValues(list).map(lambda x: (x[0], list(np.mean(np.array(x[1]), axis=0, dtype=np.int16))))

In [348]:
user_duration_day4_vec.coalesce(2).saveAsTextFile(base_path + "user_duration/day4/")

In [349]:
user_duration_day5_vec.take(2)

[('8613851429132', [60463]), ('8613913395739', [181016])]

#### server_category: 服务器业务类型

几乎都是web类型，统计没有太大意义，去掉这个特征。

In [120]:
server_category_day1_vec = qq_5000_user_dpi_day1.map(lambda x: (eval(x)[2], eval(x)[3])).groupByKey().mapValues(set)

In [121]:
server_category_day1_vec.take(3)

[('14.17.19.77', {'web'}),
 ('111.10.27.180', {'web'}),
 ('111.30.132.109', {'web'})]

#### server_diversity: 服务器每天跟几个qq用户发生连接

In [122]:
server_diversity_day1_vec = qq_5000_user_dpi_day1.map(lambda x: (eval(x)[2], eval(x)[1])).groupByKey().mapValues(set)\
.map(lambda x: (x[0], [len(x[1])]))
server_diversity_day2_vec = qq_5000_user_dpi_day2.map(lambda x: (eval(x)[2], eval(x)[1])).groupByKey().mapValues(set)\
.map(lambda x: (x[0], [len(x[1])]))
server_diversity_day3_vec = qq_5000_user_dpi_day3.map(lambda x: (eval(x)[2], eval(x)[1])).groupByKey().mapValues(set)\
.map(lambda x: (x[0], [len(x[1])]))

In [316]:
server_diversity_day5_vec = qq_5000_user_dpi_day4.map(lambda x: (eval(x)[2], eval(x)[1])).groupByKey().mapValues(set)\
.map(lambda x: (x[0], [len(x[1])]))

In [317]:
server_diversity_day5_vec.coalesce(2).saveAsTextFile(base_path + "server_diversity/day5/")

In [123]:
server_diversity_day1_vec.take(2)

[('14.17.19.77', [192]), ('111.10.27.180', [12])]

In [426]:
server_diversity_day4_vec = sc.union([server_diversity_day1_vec, server_diversity_day2_vec, server_diversity_day3_vec])\
.groupByKey().mapValues(list).map(lambda x: (x[0], [int(sum([i for j in x[1] for i in j])/3)]))

In [427]:
server_diversity_day4_vec.take(2)

[('111.10.27.180', [10]), ('117.135.148.48', [4])]

In [428]:
server_diversity_day4_vec.coalesce(2).saveAsTextFile(base_path + "server_diversity/day4/")

In [126]:
server_diversity_day1_vec.coalesce(2).saveAsTextFile(base_path + "server_diversity/day1/")
server_diversity_day2_vec.coalesce(2).saveAsTextFile(base_path + "server_diversity/day2/")
server_diversity_day3_vec.coalesce(2).saveAsTextFile(base_path + "server_diversity/day3/")

#### server_service_ID: 大类业务ID; 统计的是每类业务的记录数

In [129]:
all_server_service_ID = list(set(qq_5000_user_dpi_3days.map(lambda x: eval(x)[10]).collect()))
print(all_server_service_ID)

['24', '18', '14', '10', '3', '2', '7', '0', '4', '23', '1', '8', '6', '21']


In [437]:
print(len(all_server_service_ID))

14


In [230]:
server_service_ID_day1_vec = qq_5000_user_dpi_day1.map(lambda x: (eval(x)[2], eval(x)[10])).groupByKey().mapValues(list)\
.map(lambda x: (x[0], dict(Counter(x[1])))).map(lambda x: (x[0], get_feature(x[1], all_server_service_ID)))
server_service_ID_day2_vec = qq_5000_user_dpi_day2.map(lambda x: (eval(x)[2], eval(x)[10])).groupByKey().mapValues(list)\
.map(lambda x: (x[0], dict(Counter(x[1])))).map(lambda x: (x[0], get_feature(x[1], all_server_service_ID)))
server_service_ID_day3_vec = qq_5000_user_dpi_day3.map(lambda x: (eval(x)[2], eval(x)[10])).groupByKey().mapValues(list)\
.map(lambda x: (x[0], dict(Counter(x[1])))).map(lambda x: (x[0], get_feature(x[1], all_server_service_ID)))

In [318]:
server_service_ID_day5_vec = qq_5000_user_dpi_day4.map(lambda x: (eval(x)[2], eval(x)[10])).groupByKey().mapValues(list)\
.map(lambda x: (x[0], dict(Counter(x[1])))).map(lambda x: (x[0], get_feature(x[1], all_server_service_ID)))

In [319]:
server_service_ID_day5_vec.coalesce(2).saveAsTextFile(base_path + "server_service_ID/day5/")

In [231]:
server_service_ID_day3_vec.take(3)

[('14.17.19.77', [0, 0, 0, 0, 0, 0, 66, 0, 199, 0, 0, 0, 0, 0]),
 ('111.10.27.180', [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 12, 0]),
 ('111.30.132.109', [0, 0, 0, 0, 0, 0, 0, 0, 320, 0, 0, 0, 2, 0])]

In [423]:
server_service_ID_day4_vec = sc.union([server_service_ID_day1_vec, server_service_ID_day2_vec, server_service_ID_day3_vec])\
.groupByKey().mapValues(list).map(lambda x: (x[0], [int(i/3) for i in np.sum(x[1], axis=0)]))

In [424]:
server_service_ID_day4_vec.take(3)

[('111.10.27.180', [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 17, 0]),
 ('117.135.148.48', [0, 0, 0, 0, 0, 0, 0, 0, 39, 0, 0, 0, 1, 0]),
 ('122.228.70.172', [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0])]

In [425]:
server_service_ID_day4_vec.coalesce(2).saveAsTextFile(base_path + "server_service_ID/day4/")

In [234]:
server_service_ID_day1_vec.coalesce(2).saveAsTextFile(base_path + "server_service_ID/day1/")
server_service_ID_day2_vec.coalesce(2).saveAsTextFile(base_path + "server_service_ID/day2/")
server_service_ID_day3_vec.coalesce(2).saveAsTextFile(base_path + "server_service_ID/day3/")

#### server_service_id: 小类业务id

In [135]:
all_server_service_id = list(set(qq_5000_user_dpi_3days.map(lambda x: eval(x)[11]).collect()))
print(all_server_service_id)

['2091', '14', '782', '1844', '2087', '1035', '1544', '1049', '1029', '2346', '2171', '2573', '258', '1827', '2082', '337', '1043', '2148', '2088', '1793', '1545', '339', '515', '0', '1553', '1050', '2570', '2149', '323', '1326', '1073', '1025', '1042', '1322', '1041', '528', '1072', '322', '1074', '802', '777']


In [436]:
print(len(all_server_service_id))

41


In [235]:
server_service_id_day1_vec = qq_5000_user_dpi_day1.map(lambda x: (eval(x)[2], eval(x)[11])).groupByKey().mapValues(list)\
.map(lambda x: (x[0], dict(Counter(x[1])))).map(lambda x: (x[0], get_feature(x[1], all_server_service_id)))
server_service_id_day2_vec = qq_5000_user_dpi_day2.map(lambda x: (eval(x)[2], eval(x)[11])).groupByKey().mapValues(list)\
.map(lambda x: (x[0], dict(Counter(x[1])))).map(lambda x: (x[0], get_feature(x[1], all_server_service_id)))
server_service_id_day3_vec = qq_5000_user_dpi_day3.map(lambda x: (eval(x)[2], eval(x)[11])).groupByKey().mapValues(list)\
.map(lambda x: (x[0], dict(Counter(x[1])))).map(lambda x: (x[0], get_feature(x[1], all_server_service_id)))

In [320]:
server_service_id_day5_vec = qq_5000_user_dpi_day4.map(lambda x: (eval(x)[2], eval(x)[11])).groupByKey().mapValues(list)\
.map(lambda x: (x[0], dict(Counter(x[1])))).map(lambda x: (x[0], get_feature(x[1], all_server_service_id)))

In [321]:
server_service_id_day5_vec.coalesce(2).saveAsTextFile(base_path + "server_service_id/day5/")

In [236]:
print(server_service_id_day1_vec.take(2))

[('14.17.19.77', [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 75, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 193, 0, 0, 0, 0, 0, 0, 0, 0, 0]), ('111.10.27.180', [0, 0, 0, 0, 0, 0, 0, 0, 18, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])]


In [420]:
server_service_id_day4_vec = sc.union([server_service_id_day1_vec, server_service_id_day2_vec, server_service_id_day3_vec])\
.groupByKey().mapValues(list).map(lambda x: (x[0], [int(i/3) for i in np.sum(x[1], axis=0)]))

In [421]:
print(server_service_id_day4_vec.take(2))

[('111.10.27.180', [0, 0, 0, 0, 0, 0, 0, 0, 17, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]), ('117.135.148.48', [0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 39, 0, 0, 0, 0, 0, 0, 0, 0, 0])]


In [422]:
server_service_id_day4_vec.coalesce(2).saveAsTextFile(base_path + "server_service_id/day4/")

In [239]:
server_service_id_day1_vec.coalesce(2).saveAsTextFile(base_path + "server_service_id/day1/")
server_service_id_day2_vec.coalesce(2).saveAsTextFile(base_path + "server_service_id/day2/")
server_service_id_day3_vec.coalesce(2).saveAsTextFile(base_path + "server_service_id/day3/")

#### server_active_time: 服务器活跃时间，每天24小时每小时的记录数

In [143]:
server_activetime_day1_vec = qq_5000_user_dpi_day1.map(lambda x: (eval(x)[2], eval(x)[0][-8:-6])).groupByKey().mapValues(list)\
.map(lambda x: (x[0], dict(Counter(x[1])))).map(lambda x: (x[0], get_feature(x[1], time_hour)))
server_activetime_day2_vec = qq_5000_user_dpi_day2.map(lambda x: (eval(x)[2], eval(x)[0][-8:-6])).groupByKey().mapValues(list)\
.map(lambda x: (x[0], dict(Counter(x[1])))).map(lambda x: (x[0], get_feature(x[1], time_hour)))
server_activetime_day3_vec = qq_5000_user_dpi_day3.map(lambda x: (eval(x)[2], eval(x)[0][-8:-6])).groupByKey().mapValues(list)\
.map(lambda x: (x[0], dict(Counter(x[1])))).map(lambda x: (x[0], get_feature(x[1], time_hour)))

In [322]:
server_activetime_day5_vec = qq_5000_user_dpi_day4.map(lambda x: (eval(x)[2], eval(x)[0][-8:-6])).groupByKey().mapValues(list)\
.map(lambda x: (x[0], dict(Counter(x[1])))).map(lambda x: (x[0], get_feature(x[1], time_hour)))

In [323]:
server_activetime_day5_vec.coalesce(2).saveAsTextFile(base_path + "server_activetime/day5/")

In [145]:
print(server_activetime_day2_vec.take(3))

[('14.17.19.77', [14, 3, 0, 1, 6, 5, 3, 6, 9, 9, 12, 12, 5, 11, 5, 8, 10, 9, 13, 20, 15, 22, 19, 29]), ('111.10.27.180', [0, 0, 0, 3, 3, 0, 1, 3, 0, 1, 0, 0, 1, 4, 1, 1, 1, 0, 3, 1, 0, 0, 0, 0]), ('111.30.132.109', [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3, 0, 0, 0, 0, 0])]


In [417]:
server_activetime_day4_vec = sc.union([server_activetime_day1_vec, server_activetime_day2_vec, server_activetime_day3_vec])\
.groupByKey().mapValues(list).map(lambda x: (x[0], [int(i/3) for i in np.sum(x[1], axis=0)]))

In [418]:
server_activetime_day4_vec.take(2)

[('111.10.27.180',
  [0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 2, 0, 0, 2, 0, 0, 0, 0, 2, 0, 0, 0, 1, 0]),
 ('117.135.148.48',
  [2, 2, 1, 0, 1, 0, 0, 0, 1, 0, 1, 4, 0, 0, 0, 1, 0, 0, 0, 7, 4, 2, 4, 5])]

In [419]:
server_activetime_day4_vec.coalesce(2).saveAsTextFile(base_path + "server_activetime/day4/")

In [147]:
server_activetime_day1_vec.coalesce(2).saveAsTextFile(base_path + "server_activetime/day1/")
server_activetime_day2_vec.coalesce(2).saveAsTextFile(base_path + "server_activetime/day2/")
server_activetime_day3_vec.coalesce(2).saveAsTextFile(base_path + "server_activetime/day3/")

#### server_indegree: 服务器入度，每天跟用户发生连接的次数

In [148]:
server_indegree_day1_vec = qq_5000_user_dpi_day1.map(lambda x: (eval(x)[2], eval(x)[1])).groupByKey().mapValues(list)\
.map(lambda x: (x[0], [len(x[1])]))
server_indegree_day2_vec = qq_5000_user_dpi_day2.map(lambda x: (eval(x)[2], eval(x)[1])).groupByKey().mapValues(list)\
.map(lambda x: (x[0], [len(x[1])]))
server_indegree_day3_vec = qq_5000_user_dpi_day3.map(lambda x: (eval(x)[2], eval(x)[1])).groupByKey().mapValues(list)\
.map(lambda x: (x[0], [len(x[1])]))

In [324]:
server_indegree_day5_vec = qq_5000_user_dpi_day4.map(lambda x: (eval(x)[2], eval(x)[1])).groupByKey().mapValues(list)\
.map(lambda x: (x[0], [len(x[1])]))

In [325]:
server_indegree_day5_vec.coalesce(2).saveAsTextFile(base_path + "server_indegree/day5/")

In [149]:
server_indegree_day2_vec.take(2)

[('14.17.19.77', [246]), ('111.10.27.180', [23])]

In [414]:
server_indegree_day4_vec = sc.union([server_indegree_day1_vec, server_indegree_day2_vec, server_indegree_day3_vec])\
.groupByKey().mapValues(list).map(lambda x: (x[0], [int(sum([i for j in x[1] for i in j])/3)]))

In [415]:
server_indegree_day4_vec.coalesce(2).saveAsTextFile(base_path + "server_indegree/day4/")

In [151]:
server_indegree_day1_vec.coalesce(2).saveAsTextFile(base_path + "server_indegree/day1/")
server_indegree_day2_vec.coalesce(2).saveAsTextFile(base_path + "server_indegree/day2/")
server_indegree_day3_vec.coalesce(2).saveAsTextFile(base_path + "server_indegree/day3/")

#### server_in_strength: 服务器每天下行报文数

In [154]:
server_in_strength_day1_vec = qq_5000_user_dpi_day1.map(lambda x: (eval(x)[2], int(eval(x)[8]))).groupByKey().mapValues(list)\
.map(lambda x: (x[0], [sum(x[1])]))
server_in_strength_day2_vec = qq_5000_user_dpi_day2.map(lambda x: (eval(x)[2], int(eval(x)[8]))).groupByKey().mapValues(list)\
.map(lambda x: (x[0], [sum(x[1])]))
server_in_strength_day3_vec = qq_5000_user_dpi_day3.map(lambda x: (eval(x)[2], int(eval(x)[8]))).groupByKey().mapValues(list)\
.map(lambda x: (x[0], [sum(x[1])]))

In [326]:
server_in_strength_day5_vec = qq_5000_user_dpi_day4.map(lambda x: (eval(x)[2], int(eval(x)[8]))).groupByKey().mapValues(list)\
.map(lambda x: (x[0], [sum(x[1])]))

In [327]:
server_in_strength_day5_vec.coalesce(2).saveAsTextFile(base_path + "server_in_strength/day5/")

In [155]:
server_in_strength_day1_vec.take(2)

[('14.17.19.77', [2235]), ('111.10.27.180', [983])]

In [412]:
server_in_strength_day4_vec = sc.union([server_in_strength_day1_vec, server_in_strength_day2_vec, server_in_strength_day3_vec])\
.groupByKey().mapValues(list).map(lambda x: (x[0], [int(sum([i for j in x[1] for i in j])/3)]))

In [413]:
server_in_strength_day4_vec.coalesce(2).saveAsTextFile(base_path + "server_in_strength/day4/")

In [157]:
server_in_strength_day1_vec.coalesce(2).saveAsTextFile(base_path + "server_in_strength/day1/")
server_in_strength_day2_vec.coalesce(2).saveAsTextFile(base_path + "server_in_strength/day2/")
server_in_strength_day3_vec.coalesce(2).saveAsTextFile(base_path + "server_in_strength/day3/")

#### server_out_strength: 服务器上行报文数

In [158]:
server_out_strength_day1_vec = qq_5000_user_dpi_day1.map(lambda x: (eval(x)[2], int(eval(x)[9]))).groupByKey().mapValues(list)\
.map(lambda x: (x[0], [sum(x[1])]))
server_out_strength_day2_vec = qq_5000_user_dpi_day2.map(lambda x: (eval(x)[2], int(eval(x)[9]))).groupByKey().mapValues(list)\
.map(lambda x: (x[0], [sum(x[1])]))
server_out_strength_day3_vec = qq_5000_user_dpi_day3.map(lambda x: (eval(x)[2], int(eval(x)[9]))).groupByKey().mapValues(list)\
.map(lambda x: (x[0], [sum(x[1])]))

In [328]:
server_out_strength_day5_vec = qq_5000_user_dpi_day4.map(lambda x: (eval(x)[2], int(eval(x)[9]))).groupByKey().mapValues(list)\
.map(lambda x: (x[0], [sum(x[1])]))

In [329]:
server_out_strength_day5_vec.coalesce(2).saveAsTextFile(base_path + "server_out_strength/day5/")

In [409]:
server_out_strength_day4_vec.take(2)

[('111.10.27.180', [1506]), ('117.135.148.48', [374])]

In [407]:
server_out_strength_day4_vec = sc.union([server_out_strength_day1_vec, server_out_strength_day2_vec, server_out_strength_day3_vec])\
.groupByKey().mapValues(list).map(lambda x: (x[0], [int(sum([i for j in x[1] for i in j])/3)]))

In [410]:
server_out_strength_day4_vec.coalesce(2).saveAsTextFile(base_path + "server_out_strength/day4/")

In [161]:
server_out_strength_day1_vec.coalesce(2).saveAsTextFile(base_path + "server_out_strength/day1/")
server_out_strength_day2_vec.coalesce(2).saveAsTextFile(base_path + "server_out_strength/day2/")
server_out_strength_day3_vec.coalesce(2).saveAsTextFile(base_path + "server_out_strength/day3/")

#### server_duration: 每个qq服务器每天与qq用户之间的连接总时长

In [350]:
server_duration_day1_vec = qq_5000_user_dpi_day1.map(lambda x: (eval(x)[2], int(eval(x)[7]))).groupByKey().mapValues(list)\
.map(lambda x: (x[0], [sum(x[1])]))
server_duration_day2_vec = qq_5000_user_dpi_day2.map(lambda x: (eval(x)[2], int(eval(x)[7]))).groupByKey().mapValues(list)\
.map(lambda x: (x[0], [sum(x[1])]))
server_duration_day3_vec = qq_5000_user_dpi_day3.map(lambda x: (eval(x)[2], int(eval(x)[7]))).groupByKey().mapValues(list)\
.map(lambda x: (x[0], [sum(x[1])]))

In [351]:
server_duration_day5_vec = qq_5000_user_dpi_day4.map(lambda x: (eval(x)[2], int(eval(x)[7]))).groupByKey().mapValues(list)\
.map(lambda x: (x[0], [sum(x[1])]))

In [402]:
server_duration_day4_vec = sc.union([server_duration_day1_vec, server_duration_day2_vec, server_duration_day3_vec])\
.groupByKey().mapValues(list).map(lambda x: (x[0], [int(sum([i for j in x[1] for i in j])/3)]))

In [411]:
server_duration_day4_vec.coalesce(2).saveAsTextFile(base_path + "server_duration/day4/")

In [353]:
server_duration_day1_vec.coalesce(2).saveAsTextFile(base_path + "server_duration/day1/")
server_duration_day2_vec.coalesce(2).saveAsTextFile(base_path + "server_duration/day2/")
server_duration_day3_vec.coalesce(2).saveAsTextFile(base_path + "server_duration/day3/")
server_duration_day5_vec.coalesce(2).saveAsTextFile(base_path + "server_duration/day5/")

In [397]:
list(np.mean(np.array([[1],[3],[5]]), axis=0, dtype=np.int16))

[3]

In [404]:
server_duration_day4_vec.take(2)

[('111.10.27.180', [222873]), ('117.135.148.48', [264432])]

## 边的信息去掉。。
#### edge_duration: 用户和服务器之间连接时长  

第4天边的信息怎么处理？根据前三天的取平均？？

In [164]:
edge_duration_day1_vec = qq_5000_user_dpi_day1.map(lambda x: ('day1-'+eval(x)[1]+'-'+eval(x)[2], int(eval(x)[7])))\
.groupByKey().mapValues(list).map(lambda x: (x[0], [sum(x[1])]))
edge_duration_day2_vec = qq_5000_user_dpi_day2.map(lambda x: ('day2-'+eval(x)[1]+'-'+eval(x)[2], int(eval(x)[7])))\
.groupByKey().mapValues(list).map(lambda x: (x[0], [sum(x[1])]))
edge_duration_day3_vec = qq_5000_user_dpi_day3.map(lambda x: ('day3-'+eval(x)[1]+'-'+eval(x)[2], int(eval(x)[7])))\
.groupByKey().mapValues(list).map(lambda x: (x[0], [sum(x[1])]))

In [163]:
edge_duration_day1_vec.take(2)

[('day1-8618795929560-120.198.199.170', [35360]),
 ('day1-8615240232099-120.198.203.152', [1877])]

In [186]:
edge_duration_day4_vec = sc.union([edge_duration_day1_vec, edge_duration_day2_vec, edge_duration_day3_vec])\
.map(lambda x: (x[0][5:], x[1])).groupByKey().mapValues(list)\
.map(lambda x: (x[0], [sum(sum(i) for i in x[1])//3]))

In [187]:
edge_duration_day4_vec.take(3)

[('8613913350889-58.250.132.244', [1278]),
 ('8615922094581-120.196.210.176', [16177]),
 ('8613813395208-117.135.169.180', [286])]

In [189]:
edge_duration_day1_vec.coalesce(24).saveAsTextFile(base_path + "edge_duration/day1/")
edge_duration_day2_vec.coalesce(24).saveAsTextFile(base_path + "edge_duration/day2/")
edge_duration_day3_vec.coalesce(24).saveAsTextFile(base_path + "edge_duration/day3/")
edge_duration_day4_vec.coalesce(24).saveAsTextFile(base_path + "edge_duration/day4/")

#### edge_uppacket: 上行报文数

In [190]:
edge_uppacket_day1_vec = qq_5000_user_dpi_day1.map(lambda x: ('day1-'+eval(x)[1]+'-'+eval(x)[2], int(eval(x)[8])))\
.groupByKey().mapValues(list).map(lambda x: (x[0], [sum(x[1])]))
edge_uppacket_day2_vec = qq_5000_user_dpi_day2.map(lambda x: ('day2-'+eval(x)[1]+'-'+eval(x)[2], int(eval(x)[8])))\
.groupByKey().mapValues(list).map(lambda x: (x[0], [sum(x[1])]))
edge_uppacket_day3_vec = qq_5000_user_dpi_day3.map(lambda x: ('day3-'+eval(x)[1]+'-'+eval(x)[2], int(eval(x)[8])))\
.groupByKey().mapValues(list).map(lambda x: (x[0], [sum(x[1])]))

In [191]:
edge_uppacket_day1_vec.take(2)

[('day1-8618795929560-120.198.199.170', [76]),
 ('day1-8615240232099-120.198.203.152', [9])]

In [192]:
edge_uppacket_day4_vec = sc.union([edge_uppacket_day1_vec, edge_uppacket_day2_vec, edge_uppacket_day3_vec])\
.map(lambda x: (x[0][5:], x[1])).groupByKey().mapValues(list)\
.map(lambda x: (x[0], [sum(sum(i) for i in x[1])//3]))

In [193]:
edge_uppacket_day4_vec.take(2)

[('8613913350889-58.250.132.244', [6]), ('8615922094581-120.196.210.176', [7])]

In [194]:
edge_uppacket_day1_vec.coalesce(24).saveAsTextFile(base_path + "edge_uppacket/day1/")
edge_uppacket_day2_vec.coalesce(24).saveAsTextFile(base_path + "edge_uppacket/day2/")
edge_uppacket_day3_vec.coalesce(24).saveAsTextFile(base_path + "edge_uppacket/day3/")
edge_uppacket_day4_vec.coalesce(24).saveAsTextFile(base_path + "edge_uppacket/day4/")

#### edge_downpacket: 下行报文数

In [195]:
edge_downpacket_day1_vec = qq_5000_user_dpi_day1.map(lambda x: ('day1-'+eval(x)[1]+'-'+eval(x)[2], int(eval(x)[9])))\
.groupByKey().mapValues(list).map(lambda x: (x[0], [sum(x[1])]))
edge_downpacket_day2_vec = qq_5000_user_dpi_day2.map(lambda x: ('day2-'+eval(x)[1]+'-'+eval(x)[2], int(eval(x)[9])))\
.groupByKey().mapValues(list).map(lambda x: (x[0], [sum(x[1])]))
edge_downpacket_day3_vec = qq_5000_user_dpi_day3.map(lambda x: ('day3-'+eval(x)[1]+'-'+eval(x)[2], int(eval(x)[9])))\
.groupByKey().mapValues(list).map(lambda x: (x[0], [sum(x[1])]))

In [196]:
edge_downpacket_day3_vec.take(2)

[('day3-8613952055100-111.30.135.151', [10]),
 ('day3-8613451804425-183.232.88.148', [970])]

In [197]:
edge_downpacket_day4_vec = sc.union([edge_downpacket_day1_vec, edge_downpacket_day2_vec, edge_downpacket_day3_vec])\
.map(lambda x: (x[0][5:], x[1])).groupByKey().mapValues(list)\
.map(lambda x: (x[0], [sum(sum(i) for i in x[1])//3]))

In [198]:
edge_downpacket_day4_vec.take(2)

[('8613913350889-58.250.132.244', [4]), ('8615922094581-120.196.210.176', [5])]

In [199]:
edge_downpacket_day1_vec.coalesce(24).saveAsTextFile(base_path + "edge_downpacket/day1/")
edge_downpacket_day2_vec.coalesce(24).saveAsTextFile(base_path + "edge_downpacket/day2/")
edge_downpacket_day3_vec.coalesce(24).saveAsTextFile(base_path + "edge_downpacket/day3/")
edge_downpacket_day4_vec.coalesce(24).saveAsTextFile(base_path + "edge_downpacket/day4/")

## merge features

#### positive pairs

In [209]:
positive_pairs_day1 = sc.parallelize(list(set(qq_5000_user_dpi_day1.map(lambda x: 'day1-'+eval(x)[1]+'-'+eval(x)[2]).collect())))
positive_pairs_day2 = sc.parallelize(list(set(qq_5000_user_dpi_day2.map(lambda x: 'day2-'+eval(x)[1]+'-'+eval(x)[2]).collect())))
positive_pairs_day3 = sc.parallelize(list(set(qq_5000_user_dpi_day3.map(lambda x: 'day3-'+eval(x)[1]+'-'+eval(x)[2]).collect())))

In [210]:
def phone_of(x):
    return x.split('-')[1]
def server_of(x):
    return x.split('-')[2]

In [240]:
#将(key, value)的形式变成字典
qq_user_phone_3digits_dict = qq_user_phone_3digits_vec.collectAsMap()
user_brand_dict = user_brand_vec.collectAsMap()

In [356]:
user_activetime_day1_dict = user_activetime_day1_vec.collectAsMap()
user_diversity_day1_dict = user_diversity_day1_vec.collectAsMap()
user_indegree_day1_dict = user_indegree_day1_vec.collectAsMap()
user_in_strength_day1_dict = user_in_strength_day1_vec.collectAsMap()
user_out_strength_day1_dict = user_out_strength_day1_vec.collectAsMap()
user_duration_day1_dict = user_duration_day1_vec.collectAsMap()
server_diversity_day1_dict = server_diversity_day1_vec.collectAsMap()
server_service_ID_day1_dict = server_service_ID_day1_vec.collectAsMap()
server_service_id_day1_dict = server_service_id_day1_vec.collectAsMap()
server_activetime_day1_dict = server_activetime_day1_vec.collectAsMap()
server_indegree_day1_dict = server_indegree_day1_vec.collectAsMap() 
server_in_strength_day1_dict = server_in_strength_day1_vec.collectAsMap()
server_out_strength_day1_dict = server_out_strength_day1_vec.collectAsMap()
server_duration_day1_dict = server_duration_day1_vec.collectAsMap()
#edge_duration_day1_dict = edge_duration_day1_vec.collectAsMap()
#edge_uppacket_day1_dict = edge_uppacket_day1_vec.collectAsMap()
#edge_downpacket_day1_dict = edge_downpacket_day1_vec.collectAsMap()

In [359]:
positive_pairs_features_day1 = positive_pairs_day1.map(lambda x: (x, [qq_user_phone_3digits_dict[phone_of(x)], \
                                                                     user_brand_dict[phone_of(x)], \
                                                                     user_activetime_day1_dict[phone_of(x)],\
                                                                     user_diversity_day1_dict[phone_of(x)], \
                                                                     user_indegree_day1_dict[phone_of(x)], \
                                                                     user_in_strength_day1_dict[phone_of(x)], \
                                                                     user_out_strength_day1_dict[phone_of(x)], \
                                                                     user_duration_day1_dict[phone_of(x)], \
                                                                     server_diversity_day1_dict[server_of(x)], \
                                                                     server_service_ID_day1_dict[server_of(x)], \
                                                                     server_service_id_day1_dict[server_of(x)], \
                                                                     server_activetime_day1_dict[server_of(x)], \
                                                                     server_indegree_day1_dict[server_of(x)], \
                                                                     server_in_strength_day1_dict[server_of(x)], \
                                                                     server_out_strength_day1_dict[server_of(x)],\
                                                                     server_duration_day1_dict[server_of(x)]]))

In [360]:
print(positive_pairs_features_day1.take(2))

[('day1-8613851906441-221.179.19.156', [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 2, 2, 0, 41, 8, 2, 9, 23, 3, 13, 2, 20, 1, 3, 12, 0, 2, 2], [42], [145], [1595], [1629], [1709003], [131], [0, 0, 0, 0, 0, 0, 147, 0, 71, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 147, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 71, 0, 0, 0, 0, 0, 0, 0, 0, 0], [10, 1, 3, 3, 5, 6, 4, 6, 13, 23, 10, 10, 7, 13, 12, 15, 19, 11, 6, 10, 6, 14, 6, 5], [218], [1253], [931], [1739364]]), ('day1-8618252050428-111.30.128.27', [[0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0,

In [361]:
len([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 2, 2, 0, 41, 8, 2, 9, 23, 3, 13, 2, 20, 1, 3, 12, 0, 2, 2], [42], [145], [1595], [1629], [1709003], [131], [0, 0, 0, 0, 0, 0, 147, 0, 71, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 147, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 71, 0, 0, 0, 0, 0, 0, 0, 0, 0], [10, 1, 3, 3, 5, 6, 4, 6, 13, 23, 10, 10, 7, 13, 12, 15, 19, 11, 6, 10, 6, 14, 6, 5], [218], [1253], [931], [1739364]])

16

In [362]:
positive_pairs_features_day1.coalesce(10).saveAsTextFile(base_path + "new_positive_pairs_features/day1/")

In [363]:
user_activetime_day2_dict = user_activetime_day2_vec.collectAsMap()
user_diversity_day2_dict = user_diversity_day2_vec.collectAsMap()
user_indegree_day2_dict = user_indegree_day2_vec.collectAsMap()
user_in_strength_day2_dict = user_in_strength_day2_vec.collectAsMap()
user_out_strength_day2_dict = user_out_strength_day2_vec.collectAsMap()
user_duration_day2_dict = user_duration_day2_vec.collectAsMap()
server_diversity_day2_dict = server_diversity_day2_vec.collectAsMap()
server_service_ID_day2_dict = server_service_ID_day2_vec.collectAsMap()
server_service_id_day2_dict = server_service_id_day2_vec.collectAsMap()
server_activetime_day2_dict = server_activetime_day2_vec.collectAsMap()
server_indegree_day2_dict = server_indegree_day2_vec.collectAsMap() 
server_in_strength_day2_dict = server_in_strength_day2_vec.collectAsMap()
server_out_strength_day2_dict = server_out_strength_day2_vec.collectAsMap()
server_duration_day2_dict = server_duration_day2_vec.collectAsMap()

In [364]:
positive_pairs_features_day2 = positive_pairs_day2.map(lambda x: (x, [qq_user_phone_3digits_dict[phone_of(x)], \
                                                                     user_brand_dict[phone_of(x)], \
                                                                     user_activetime_day2_dict[phone_of(x)],\
                                                                     user_diversity_day2_dict[phone_of(x)], \
                                                                     user_indegree_day2_dict[phone_of(x)], \
                                                                     user_in_strength_day2_dict[phone_of(x)], \
                                                                     user_out_strength_day2_dict[phone_of(x)], \
                                                                     user_duration_day2_dict[phone_of(x)], \
                                                                     server_diversity_day2_dict[server_of(x)], \
                                                                     server_service_ID_day2_dict[server_of(x)], \
                                                                     server_service_id_day2_dict[server_of(x)], \
                                                                     server_activetime_day2_dict[server_of(x)], \
                                                                     server_indegree_day2_dict[server_of(x)], \
                                                                     server_in_strength_day2_dict[server_of(x)], \
                                                                     server_out_strength_day2_dict[server_of(x)],\
                                                                     server_duration_day2_dict[server_of(x)]]))

In [365]:
positive_pairs_features_day2.coalesce(10).saveAsTextFile(base_path + "new_positive_pairs_features/day2/")

In [368]:
user_activetime_day3_dict = user_activetime_day3_vec.collectAsMap()
user_diversity_day3_dict = user_diversity_day3_vec.collectAsMap()
user_indegree_day3_dict = user_indegree_day3_vec.collectAsMap()
user_in_strength_day3_dict = user_in_strength_day3_vec.collectAsMap()
user_out_strength_day3_dict = user_out_strength_day3_vec.collectAsMap()
user_duration_day3_dict = user_duration_day3_vec.collectAsMap()
server_diversity_day3_dict = server_diversity_day3_vec.collectAsMap()
server_service_ID_day3_dict = server_service_ID_day3_vec.collectAsMap()
server_service_id_day3_dict = server_service_id_day3_vec.collectAsMap()
server_activetime_day3_dict = server_activetime_day3_vec.collectAsMap()
server_indegree_day3_dict = server_indegree_day3_vec.collectAsMap() 
server_in_strength_day3_dict = server_in_strength_day3_vec.collectAsMap()
server_out_strength_day3_dict = server_out_strength_day3_vec.collectAsMap()
server_duration_day3_dict = server_duration_day3_vec.collectAsMap()

In [369]:
positive_pairs_features_day3 = positive_pairs_day3.map(lambda x: (x, [qq_user_phone_3digits_dict[phone_of(x)], \
                                                                     user_brand_dict[phone_of(x)], \
                                                                     user_activetime_day3_dict[phone_of(x)],\
                                                                     user_diversity_day3_dict[phone_of(x)], \
                                                                     user_indegree_day3_dict[phone_of(x)], \
                                                                     user_in_strength_day3_dict[phone_of(x)], \
                                                                     user_out_strength_day3_dict[phone_of(x)], \
                                                                     user_duration_day3_dict[phone_of(x)], \
                                                                     server_diversity_day3_dict[server_of(x)], \
                                                                     server_service_ID_day3_dict[server_of(x)], \
                                                                     server_service_id_day3_dict[server_of(x)], \
                                                                     server_activetime_day3_dict[server_of(x)], \
                                                                     server_indegree_day3_dict[server_of(x)], \
                                                                     server_in_strength_day3_dict[server_of(x)], \
                                                                     server_out_strength_day3_dict[server_of(x)],\
                                                                     server_duration_day3_dict[server_of(x)]]))

In [371]:
positive_pairs_features_day3.coalesce(10).saveAsTextFile(base_path + "new_positive_pairs_features/day3/")

In [429]:
# 以下均是对前三天的取平均
user_activetime_day4_dict = user_activetime_day4_vec.collectAsMap()
user_diversity_day4_dict = user_diversity_day4_vec.collectAsMap()
user_indegree_day4_dict = user_indegree_day4_vec.collectAsMap()
user_in_strength_day4_dict = user_in_strength_day4_vec.collectAsMap()
user_out_strength_day4_dict = user_out_strength_day4_vec.collectAsMap()
user_duration_day4_dict = user_duration_day4_vec.collectAsMap()
server_diversity_day4_dict = server_diversity_day4_vec.collectAsMap()
server_service_ID_day4_dict = server_service_ID_day4_vec.collectAsMap()
server_service_id_day4_dict = server_service_id_day4_vec.collectAsMap()
server_activetime_day4_dict = server_activetime_day4_vec.collectAsMap()
server_indegree_day4_dict = server_indegree_day4_vec.collectAsMap() 
server_in_strength_day4_dict = server_in_strength_day4_vec.collectAsMap()
server_out_strength_day4_dict = server_out_strength_day4_vec.collectAsMap()
server_duration_day4_dict = server_duration_day4_vec.collectAsMap()

In [373]:
# 以下均是取自真实的第四天数据
user_activetime_day5_dict = user_activetime_day5_vec.collectAsMap()
user_diversity_day5_dict = user_diversity_day5_vec.collectAsMap()
user_indegree_day5_dict = user_indegree_day5_vec.collectAsMap()
user_in_strength_day5_dict = user_in_strength_day5_vec.collectAsMap()
user_out_strength_day5_dict = user_out_strength_day5_vec.collectAsMap()
user_duration_day5_dict = user_duration_day5_vec.collectAsMap()
server_diversity_day5_dict = server_diversity_day5_vec.collectAsMap()
server_service_ID_day5_dict = server_service_ID_day5_vec.collectAsMap()
server_service_id_day5_dict = server_service_id_day5_vec.collectAsMap()
server_activetime_day5_dict = server_activetime_day5_vec.collectAsMap()
server_indegree_day5_dict = server_indegree_day5_vec.collectAsMap() 
server_in_strength_day5_dict = server_in_strength_day5_vec.collectAsMap()
server_out_strength_day5_dict = server_out_strength_day5_vec.collectAsMap()
server_duration_day5_dict = server_duration_day5_vec.collectAsMap()

#### unlabled pairs

unlabled对的特征怎么处理？

In [259]:
user_phone_5000 = sc.textFile(base_path + "qq_user_phone_5000/part-*").collect()
server_ip_1684 = sc.textFile(base_path + "server_ip_1684/part-*").collect()

In [260]:
all_edges = [] # 5000 * 1684 = 8420000
for u in user_phone_5000:
    for s in server_ip_1684:
        edge = u + '-' + s
        all_edges.append(edge)

In [262]:
positive_pairs_3days = qq_5000_user_dpi_3days.map(lambda x: eval(x)[1] + '-' + eval(x)[2]).collect()

In [270]:
# 三天都没有过连接的user-server对作为unlabled数据
unlabled_edges = sc.parallelize(list(set(all_edges)-set(positive_pairs_3days))).map(lambda x: 'day123-' + x) # 8268837

In [271]:
print(unlabled_edges.count(), unlabled_edges.take(3))

8268837 ['day123-8615151840328-119.167.195.26', 'day123-8615850607902-203.205.161.165', 'day123-8615151857865-117.135.171.164']


In [431]:
unlabled_pairs_features = unlabled_edges.map(lambda x: (x, [qq_user_phone_3digits_dict[phone_of(x)], \
                                                             user_brand_dict[phone_of(x)], \
                                                             user_activetime_day4_dict[phone_of(x)],\
                                                             user_diversity_day4_dict[phone_of(x)], \
                                                             user_indegree_day4_dict[phone_of(x)], \
                                                             user_in_strength_day4_dict[phone_of(x)], \
                                                             user_out_strength_day4_dict[phone_of(x)], \
                                                             user_duration_day4_dict[phone_of(x)], \
                                                             server_diversity_day4_dict[server_of(x)], \
                                                             server_service_ID_day4_dict[server_of(x)], \
                                                             server_service_id_day4_dict[server_of(x)], \
                                                             server_activetime_day4_dict[server_of(x)], \
                                                             server_indegree_day4_dict[server_of(x)], \
                                                             server_in_strength_day4_dict[server_of(x)], \
                                                             server_out_strength_day4_dict[server_of(x)],\
                                                             server_duration_day4_dict[server_of(x)]]))

In [432]:
print(unlabled_pairs_features.take(2))

[('day123-8615151840328-119.167.195.26', [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [1, 0, 0, 0, 0, 1, 2, 1, 4, 1, 1, 1, 0, 1, 0, 2, 0, 2, 4, 2, 0, 0, 0, 0], [13], [32], [164], [189], [2086], [0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [1], [8], [9], [2905]]), ('day123-8615850607902-203.205.161.165', [[0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [14, 4, 9, 2, 1, 1, 0, 7, 2, 6, 9, 9, 7, 

In [433]:
unlabled_pairs_features.coalesce(50).saveAsTextFile(base_path + "new_unlabled_pairs_features/")

### Test Data

In [286]:
positive_pairs_day4 = sc.parallelize(list(set(qq_5000_user_dpi_day4.filter(lambda x: eval(x)[2] in server_ip_1684)\
                                    .map(lambda x: 'day4-' + eval(x)[1] + '-' + eval(x)[2]).collect())))

In [287]:
print(positive_pairs_day4.count(), positive_pairs_day4.take(2))

77653 ['day4-8615005274456-120.198.199.176', 'day4-8613770922916-112.25.58.180']


In [283]:
positive_pairs_day4.coalesce(10).saveAsTextFile(base_path + "test_positive_pairs/")

In [281]:
all_pairs = sc.parallelize(all_edges).map(lambda x: 'day4-' + x)

In [438]:
test_pairs_features = all_pairs.map(lambda x: (x, [qq_user_phone_3digits_dict[phone_of(x)], \
                                                 user_brand_dict[phone_of(x)], \
                                                 user_activetime_day5_dict[phone_of(x)],\
                                                 user_diversity_day5_dict[phone_of(x)], \
                                                 user_indegree_day5_dict[phone_of(x)], \
                                                 user_in_strength_day5_dict[phone_of(x)], \
                                                 user_out_strength_day5_dict[phone_of(x)], \
                                                 user_duration_day5_dict[phone_of(x)], \
                                                 server_diversity_day5_dict.get(server_of(x),[0]), \
                                                 server_service_ID_day5_dict.get(server_of(x),[0 for i in range(14)]), \
                                                 server_service_id_day5_dict.get(server_of(x),[0 for i in range(41)]), \
                                                 server_activetime_day5_dict.get(server_of(x),[0 for i in range(24)]), \
                                                 server_indegree_day5_dict.get(server_of(x),[0]), \
                                                 server_in_strength_day5_dict.get(server_of(x),[0]), \
                                                 server_out_strength_day5_dict.get(server_of(x),[0]),\
                                                 server_duration_day5_dict.get(server_of(x),[0])]))

In [439]:
print(test_pairs_features.count(), test_pairs_features.take(3))

8420000 [('day4-8615195898003-14.17.19.77', [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 1, 0, 0, 0, 0, 3, 2, 6, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [5], [12], [26], [41], [39064], [169], [0, 0, 0, 0, 0, 0, 62, 0, 179, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 62, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 179, 0, 0, 0, 0, 0, 0, 0, 0, 0], [8, 5, 2, 0, 1, 1, 3, 7, 8, 14, 9, 11, 10, 11, 13, 12, 9, 11, 11, 13, 15, 16, 27, 24], [241], [1710], [1328], [2603900]]), ('day4-8615195898003-111.10.27.180', [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 1, 0, 0,

In [440]:
test_pairs_features.coalesce(100).saveAsTextFile(base_path + "new_test_pairs_features/")